In [13]:
#Reading in the entire dataset
data=pd.read_csv("C:/opt/spark/All Transations - 2 Weeks.txt",sep='\t')
#Let us first analyze the entire dataset using pandas_profiling
import pandas_profiling
pandas_profiling.ProfileReport(data)

Number of variables,6
Number of observations,2107537
Total Missing (%),0.0%
Total size in memory,96.5 MiB
Average record size in memory,48.0 B
Numeric,3
Categorical,3
Boolean,0
Date,0
Text (Unique),0
Rejected,0


In [14]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

#Reading in only the first 50000 rows
data=pd.read_csv("C:/opt/spark/All Transations - 2 Weeks.txt",sep='\t',nrows=50000)
print(data.head(20))
print(data.columns)
data_cols=list(data.columns)



    order_number                                      l1  \
0         168266                             Power Tools   
1         123986                                  Safety   
2         158978                                Hardware   
3         449035  Electronics, Appliances, and Batteries   
4         781232                                  Motors   
5         116599                              Pneumatics   
6         701116                                  Motors   
7         555497                                  Motors   
8         282317                                  Safety   
9         644437                              Hand Tools   
10        830236                                Security   
11        483827                  HVAC and Refrigeration   
12        595763                                Cleaning   
13        834301                              Hand Tools   
14        190274                              Electrical   
15        489348                        

In [15]:
#As per the report there are 53054 duplicate rows resulting in 3334225 rows
#data.drop_duplicates()
data.duplicated().sum()

1157

In [16]:
#Let us try to find the brands that have been bought the most
groupbrand=data[['order_number','l1','l2','l3','sku','brand']].groupby(data['brand']).size().reset_index(name='counts')
groupbrand_sort=groupbrand.sort_values('counts',ascending=False)
print(groupbrand_sort)#The below table lists the brands that have been bought the most over the last 6 month period'''

      brand  counts
786    1793    6453
471    1068    2574
407     934    2011
1940   4355    1792
2082   4692    1612
109     258    1130
48      123    1112
3         9     936
758    1726     881
544    1231     821
743    1694     788
491    1113     563
1451   3271     560
265     585     512
1484   3356     452
1219   2768     448
1792   4015     434
1762   3966     393
1299   2959     380
363     830     358
1600   3612     335
575    1303     306
1596   3606     306
1182   2676     287
891    2034     277
1029   2351     257
1166   2639     251
1692   3813     246
1211   2741     238
1303   2966     233
...     ...     ...
1698   3825       1
791    1808       1
790    1803       1
1703   3837       1
787    1798       1
784    1788       1
1706   3846       1
1707   3847       1
782    1785       1
832    1917       1
833    1919       1
834    1922       1
1668   3769       1
1661   3738       1
858    1974       1
856    1971       1
1664   3759       1
1665   3760       1


In [17]:
#Items that were bought the most. sku-stock keeping unit
grouppid=data.groupby(data['sku']).size().reset_index(name='counts')
groupid_sort=grouppid.sort_values('counts',ascending=False)
print(groupid_sort)
#print(data[data['sku']==100164])

          sku  counts
25155  282471     224
25157  282473     191
15954  184171     130
25153  282469     104
4468    46844      81
11663  124763      76
8928   100164      74
25154  282470      74
25156  282472      69
11661  124761      64
20983  245233      59
11662  124762      59
13748  154210      52
11660  124760      51
22406  257442      49
9056   101138      48
22733  259957      47
3181    36288      46
11665  124765      38
20412  239434      37
26604  294369      35
14333  163901      33
11501  123562      32
4674    49014      32
8291    89482      31
13311  146892      31
11241  121886      31
20344  239165      30
17174  195572      30
3267    36756      29
...       ...     ...
11550  123837       1
11549  123834       1
11548  123831       1
11547  123829       1
11579  124095       1
11584  124119       1
11628  124555       1
11586  124123       1
11627  124554       1
11626  124534       1
11624  124529       1
11623  124493       1
11622  124442       1
11620  124

In [18]:
print(data['order_number'].nunique())

19105


In [19]:
#Data preprocessing
'''Step 1-To avoid the user cold start problem in Rec engines we will take into account only the users who have
made more than a threshold number of orders
'''
data=data[data.groupby('order_number').order_number.transform(len)>=10]#This grouping up and transforming operation takes some time
print(len(data))






12895


In [20]:
''' Step 2- One hot encode the 'l3' column(most granular product hierarchy)'''
df=[]
df.append(data)
columns=['l3']
for c in columns:
    df.append(pd.get_dummies(data[c],prefix=None,sparse=False))
data=pd.concat(df,axis=1)    
print(data.head())

    order_number                             l1  \
22        173984                       Cleaning   
37        649979                    Power Tools   
41        565917  Paint, Equipment and Supplies   
58        482732                     Hand Tools   
64        173984                       Cleaning   

                               l2                          l3     sku  brand  \
22  Paper Products and Dispensers         Paper Towels, Rolls   46841   1726   
37           Drilling Accessories             Step Drill Bits  299689   2163   
41       Spray Paints and Primers                Spray Paints  310773   3612   
58                  Marking Tools               Paint Markers   38789   4477   
64                     Floor Care  Floor Finishes and Sealers  264751   3712   

    12 Volt Accessories  3-Ring Binders  3.3 Inch Diameter Motors  \
22                    0               0                         0   
37                    0               0                         0   
41   

In [21]:
''' Step 3-Removing unnecessary columns:{'l1','l2','l3','sku','brand'} '''
data=data.drop(['l1','l2','l3','sku','brand'],axis=1)
print(data.head())


    order_number  12 Volt Accessories  3-Ring Binders  \
22        173984                    0               0   
37        649979                    0               0   
41        565917                    0               0   
58        482732                    0               0   
64        173984                    0               0   

    3.3 Inch Diameter Motors  5 X 20mm Glass and Ceramic Fuses  \
22                         0                                 0   
37                         0                                 0   
41                         0                                 0   
58                         0                                 0   
64                         0                                 0   

    A/C Refrigeration Accessories  AC Gearmotors  \
22                              0              0   
37                              0              0   
41                              0              0   
58                              0              0   


In [22]:
#Step 4- Cosolidate each order into a single record
data_cols=list(data.columns)
data_cols.remove('order_number')
data=data.groupby('order_number').sum()[data_cols].reset_index()
print(data.head())

   order_number  12 Volt Accessories  3-Ring Binders  \
0         68137                    0               0   
1         68476                    0               0   
2         70314                    0               0   
3         71592                    0               0   
4         74059                    0               0   

   3.3 Inch Diameter Motors  5 X 20mm Glass and Ceramic Fuses  \
0                         0                                 1   
1                         0                                 0   
2                         0                                 0   
3                         0                                 0   
4                         0                                 0   

   A/C Refrigeration Accessories  AC Gearmotors  \
0                              0              0   
1                              0              0   
2                              0              0   
3                              0              0   
4                

In [23]:
# Splitting columns into user and item columns.we can use them to build the user-item matrix
user_col='order_number'
item_cols=list(data.columns)
item_cols.remove(user_col)
print(user_col)
print(item_cols)



order_number
['12 Volt Accessories', '3-Ring Binders', '3.3 Inch Diameter Motors', '5 X 20mm Glass and Ceramic Fuses', 'A/C Refrigeration Accessories', 'AC Gearmotors', 'Abrasive Bristle Disc-Cup Shaped', 'Abrasive Cut-Off and Chop Wheels', 'Abrasive Radial Bristle Brushes', 'Abrasive Roll Kits', 'Abrasive Rolls', 'Abrasive Wheel Adapters', 'Absorbent Pads', 'Absorbent Rolls', 'Absorbent Socks and Booms', 'Additives', 'Adhesive Bandages', 'Adhesive Primers and Activators', 'Adhesive and Gasket Removers', 'Adhesives', 'Adjustable Wrench Sets', 'Adjustable Wrenches', 'Aerosol Can Grips', 'Aerosol Dusters', 'Air Circulators', 'Air Cleaner Filters', 'Air Compressor Motors', 'Air Compressor and Air Tool Oil', 'Air Cylinder Speed Controls', 'Air Cylinders', 'Air Die Grinders', 'Air Freshener Dispensers', 'Air Freshener Refills', 'Air Gun Nozzles', 'Air Guns', 'Air Hammer Chisel Retainers', 'Air Hammers', 'Air Hoses', 'Air Impact Wrenches', 'Air Motors', 'Air Operated Pumps', 'Air Ratchets', 

In [24]:
#Building the recommendation system
#Step-1 Creating similarity matrix
labels=item_cols
similarity_matrix=pd.DataFrame(index=labels,columns=labels)
#print(similarity_matrix.head())


def jaccard_similarity(x,y):
    #'''returns jaccard similarity/correlation between x and y: |intersection(x,y)|/|union(x,y)|'''
    nonzero_x = set(np.nonzero(x)[0])
    nonzero_y = set(np.nonzero(y)[0])
    intersection_size = len(nonzero_x.intersection(nonzero_y))
    union_size = len(nonzero_x.union(nonzero_y))
    if union_size == 0:
        return 0
    else:
        return intersection_size/union_size

    
    

#Filling the similarity matrix
item_df=data[item_cols]
#print(item_df)
for i in range(similarity_matrix.shape[0]):
    similarity_matrix.ix[i,i]=1.0
    x=item_df.ix[:,i]
    for j in range(i,similarity_matrix.shape[1]):
        y=item_df.ix[:,j]
        similarity_matrix.ix[i,j]=jaccard_similarity(x,y)
        similarity_matrix.ix[j,i]=similarity_matrix.ix[i,j]
              
        
print(similarity_matrix.shape)       
        

C:\Users\nitesh\Anaconda3\lib\site-packages\ipykernel_launcher.py:26: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate_ix
C:\Users\nitesh\Anaconda3\lib\site-packages\ipykernel_launcher.py:27: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate_ix


(1417, 1417)


In [25]:
#Step-2 Creating the score matrix for each user
users=data.loc[:,user_col]#Grabbing all users(all order_id's) from the data by default
columns=[user_col]+list(item_cols)
user_data=data.loc[:,columns].set_index(user_col)
user_scores=pd.DataFrame(index=users,columns=item_cols)
user_scores=data[item_cols].dot(similarity_matrix)





In [26]:
print(user_scores)

    12 Volt Accessories 3-Ring Binders 3.3 Inch Diameter Motors  \
0                     0              0                        0   
1                     0              0                        0   
2                     0              0                        0   
3                     0      0.0285714                0.0285714   
4                     0              0                        0   
5                     0              0                        0   
6                     0              0                        0   
7                     0              0                        0   
8                     0              0                        0   
9                     0              0                        0   
10                    0              0                        0   
11                    0              0                        0   
12                    0              0                        0   
13                    0              0                        

In [28]:
'''generates item ratings for users passed in from external data set and stores result as self.user_scores'''
cols = [user_col] + list(item_cols)
user_scores = pd.DataFrame(index=user_data.index, columns=item_cols)
user_scores = user_data.loc[item_cols].dot(similarity_matrix)

KeyError: 'None of [[\'12 Volt Accessories\', \'3-Ring Binders\', \'3.3 Inch Diameter Motors\', \'5 X 20mm Glass and Ceramic Fuses\', \'A/C Refrigeration Accessories\', \'AC Gearmotors\', \'Abrasive Bristle Disc-Cup Shaped\', \'Abrasive Cut-Off and Chop Wheels\', \'Abrasive Radial Bristle Brushes\', \'Abrasive Roll Kits\', \'Abrasive Rolls\', \'Abrasive Wheel Adapters\', \'Absorbent Pads\', \'Absorbent Rolls\', \'Absorbent Socks and Booms\', \'Additives\', \'Adhesive Bandages\', \'Adhesive Primers and Activators\', \'Adhesive and Gasket Removers\', \'Adhesives\', \'Adjustable Wrench Sets\', \'Adjustable Wrenches\', \'Aerosol Can Grips\', \'Aerosol Dusters\', \'Air Circulators\', \'Air Cleaner Filters\', \'Air Compressor Motors\', \'Air Compressor and Air Tool Oil\', \'Air Cylinder Speed Controls\', \'Air Cylinders\', \'Air Die Grinders\', \'Air Freshener Dispensers\', \'Air Freshener Refills\', \'Air Gun Nozzles\', \'Air Guns\', \'Air Hammer Chisel Retainers\', \'Air Hammers\', \'Air H

In [29]:
'''generates top 5 recommendations for users and stores result as self.recs'''
from tqdm import tqdm#Progress bar in python
users = data.loc[:,user_col]# grab all users in data by default
num_recs=5 # Number of recommendations
cols = ['Rec ' + str(x) for x in range(1,num_recs+1)] + ['Score ' + str(x) for x in range(1,num_recs+1)]
recs = pd.DataFrame(index=users, columns=cols)#Create a datfarame with index as users(order_id) and columns as 'Rec1,Rec2..Score1,Score2..'
progress_bar = tqdm(total = len(users), mininterval=5)#Creates a progress bar
for user in users:#Iterate through the rows
    progress_bar.update()
    sorted_items = user_scores.sort_values(by=user, ascending=False, axis=1).loc[user,:].index #Sort the items by user column(order_id)
    for i in range(num_recs):
        item = sorted_items[i]
        item_col = cols[i]
        score_col = cols[i+num_recs]# Score column starts after Recommendation column,hence i +num_recs
        recs.loc[user, item_col] = item
        recs.loc[user, score_col] = user_scores.loc[user, item]
recs.reset_index(inplace=True, drop=False)



100%|████████████████████████████████████████████████████████████████████████████████| 734/734 [00:30<00:00, 44.51it/s]

In [30]:
print(recs)

     order_number                Rec 1           Rec 2  \
0           68137  12 Volt Accessories  3-Ring Binders   
1           68476  12 Volt Accessories  3-Ring Binders   
2           70314  12 Volt Accessories  3-Ring Binders   
3           71592  12 Volt Accessories  3-Ring Binders   
4           74059  12 Volt Accessories  3-Ring Binders   
5           91461  12 Volt Accessories  3-Ring Binders   
6           95590  12 Volt Accessories  3-Ring Binders   
7           98851  12 Volt Accessories  3-Ring Binders   
8           99261  12 Volt Accessories  3-Ring Binders   
9           99755  12 Volt Accessories  3-Ring Binders   
10         100504  12 Volt Accessories  3-Ring Binders   
11         100529  12 Volt Accessories  3-Ring Binders   
12         101016  12 Volt Accessories  3-Ring Binders   
13         107482  12 Volt Accessories  3-Ring Binders   
14         109404  12 Volt Accessories  3-Ring Binders   
15         109991  12 Volt Accessories  3-Ring Binders   
16         110

[734 rows x 11 columns]


In [ ]:
recs.to_csv("C:/opt/spark/recsys.csv")